In [1]:
import main
import scrap_boxoffice as sb

In [6]:
df_movies = main.load_tsv_data("title.basics.tsv")
df_movies = main.filter_movies_after_year(df_movies, 1980)
df_ratings = main.load_tsv_data("title.ratings.tsv")
df_w_ratings = df_movies.merge(df_ratings, on='tconst', how='left')
df = main.filter_most_rated(df_w_ratings, 50000)
print(len(df))

/Users/guilhermemoreira/git/mineracao-filmes/main.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')


11574370 linhas carregadas do arquivo title.basics.tsv
      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short            Poor Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

  isAdult startYear endYear runtimeMinutes                    genres  
0       0      1894      \N              1         Documentary,Short  
1       0      1892      \N              5           Animation,Short  
2       0      1892      \N              5  Animation,Comedy,Romance  
3       0      1892      \N             12           Animation,Short  
4       0      1893      \N              1                     Short  
Filmes: 711500 linhas filtradas
Filmes não adultos: 711475 linhas filtrada

In [7]:
df['numVotes'].describe()

count    4.070000e+03
mean     2.046358e+05
std      2.461848e+05
min      5.001300e+04
25%      7.396625e+04
50%      1.176565e+05
75%      2.265220e+05
max      3.028930e+06
Name: numVotes, dtype: float64

In [10]:
teste = df.head().copy()

def insert(db, column, index, value, prefix=None):
    if prefix:
        column = f"{prefix}_{column}"
    # Adicione a coluna se ela ainda não existir
    if column not in db.columns:
        db[column] = None
    # Defina o valor para este filme específico
    db.at[index, column] = value

for index, item in teste.iterrows():
    print(item['originalTitle'])

    # Coletando dados do OMDB
    omdb_info = main.get_movie_info_from_omdb(item['tconst'])
    if omdb_info:
        for k, v in omdb_info.items():
            insert(teste, k, index, v, prefix="omdb")
        print("\tOMDB: info coletada")
    else:
        print("\tOMDB: nenhum dado retornado")
    
    # Coletando dados financeiros
    title = item['originalTitle'].replace("&", "and")
    title = title.replace("'", "")
    title = title.replace("!", "")
    title = title.replace(":", "")
    title = title.replace(",", "")
    title = title.replace("?", "")

    url = sb.get_url_title(title)
    financial_data = sb.fetch_movie_financials(url)
    if financial_data:
        for k, v in financial_data.items():
            insert(teste, k, index, v, prefix="box_office")
        print("\tFinancial: info coletada")
    else:
        print("\tFinancial: nenhum dado retornado")


Kate & Leopold


INFO:scrap_boxoffice:Acessando URL: https://www.the-numbers.com/search?searchterm=Kate+and+Leopold#tab=summary


	OMDB: info coletada


INFO:scrap_boxoffice:Tentando encontrar na página de busca.
INFO:scrap_boxoffice:Resultados de busca encontrados.
INFO:scrap_boxoffice:Acessando URL: https://www.the-numbers.com//movie/Kate-and-Leopold-(2001)#tab=summary


	Financial: info coletada
Cannibal Holocaust


...

	Financial: info coletada
The Blues Brothers


INFO:scrap_boxoffice:Acessando URL: https://www.the-numbers.com/search?searchterm=The+Blues+Brothers#tab=summary


	OMDB: info coletada


INFO:scrap_boxoffice:Tentando encontrar na página de busca.
INFO:scrap_boxoffice:Resultados de busca encontrados.
INFO:scrap_boxoffice:Acessando URL: https://www.the-numbers.com//movie/Blues-Brothers-The#tab=summary


	Financial: info coletada


In [11]:
teste.to_csv("teste.csv", sep=";", index=False)

teste.head()

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,...,omdb_Response,box_office_Domestic Box Office,box_office_International Box Office,box_office_Worldwide Box Office,box_office_Opening Weekend:,box_office_Legs:,box_office_Domestic Share:,box_office_Production Budget:,box_office_Theater counts:,box_office_Infl. Adj. Dom. BO
0,2,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,\N,118,"Comedy,Fantasy,Romance",...,True,"$47,095,453","$23,842,325","$70,937,778","$9,725,408 (20.7% of total gross)",4.84 (domestic box office/biggest weekend),66.4% (domestic box office/worldwide),"$48,000,000 (worldwide box office is 1.5 times...","2,459 opening theaters/2,467 max. theaters, 5....","$93,256,073"
1,135,tt0078935,movie,Cannibal Holocaust,Cannibal Holocaust,0,1980,\N,95,"Adventure,Horror",...,True,n/a,None,None,None,None,None,None,None,None
2,323,tt0080339,movie,Airplane!,Airplane!,0,1980,\N,88,Comedy,...,True,"$83,453,539",$768,"$83,454,307",None,None,100.0% (domestic box office/worldwide),"$3,500,000 (worldwide box office is 23.8 times...",None,"$350,877,142"
3,396,tt0080453,movie,The Blue Lagoon,The Blue Lagoon,0,1980,\N,104,"Adventure,Drama,Romance",...,True,"$47,923,795",None,None,None,174.24 (domestic box office/biggest weekend),100.0% (domestic box office/worldwide),"$4,500,000 (worldwide box office is 10.6 times...",None,"$201,493,723"
4,398,tt0080455,movie,The Blues Brothers,The Blues Brothers,0,1980,\N,133,"Adventure,Comedy,Crime",...,True,"$57,229,890","$16,570","$57,246,460","$4,858,152 (8.5% of total gross)",11.78 (domestic box office/biggest weekend),100.0% (domestic box office/worldwide),"$27,000,000 (worldwide box office is 2.1 times...","594 opening theaters/594 max. theaters, 1.0 we...","$240,620,838"


In [14]:
from progressbar import progressbar
from time import sleep

for i in progressbar(range(100), redirect_stdout=True):
    print('Some text', i)
    sleep(0.1)

  0% (0 of 100) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  1% (1 of 100) |                        | Elapsed Time: 0:00:00 ETA:   0:00:10
  2% (2 of 100) |                        | Elapsed Time: 0:00:00 ETA:   0:00:10
  3% (3 of 100) |                        | Elapsed Time: 0:00:00 ETA:   0:00:10
  4% (4 of 100) |                        | Elapsed Time: 0:00:00 ETA:   0:00:10
  5% (5 of 100) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:10
  6% (6 of 100) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:09
  7% (7 of 100) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:09
  8% (8 of 100) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:09
  9% (9 of 100) |##                      | Elapsed Time: 0:00:00 ETA:   0:00:09
 10% (10 of 100) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:09
 11% (11 of 100) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:09
 12% (12 of 100) |##                    

In [28]:
# Pós processamento
import pandas as pd

df = pd.read_csv("movies.csv", sep=";")
print(df.columns)
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,...,omdb_Response,box_office_Domestic Box Office,box_office_International Box Office,box_office_Worldwide Box Office,box_office_Opening Weekend:,box_office_Legs:,box_office_Domestic Share:,box_office_Production Budget:,box_office_Theater counts:,box_office_Infl. Adj. Dom. BO
0,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142,Drama,9.3,...,True,"$28,313,053","$471,884","$28,784,937","$727,327 (2.6% of total gross)",11.78 (domestic box office/biggest weekend),98.4% (domestic box office/worldwide),"$25,000,000 (worldwide box office is 1.2 times...","33 opening theaters/972 max. theaters, 11.5 we...","$76,238,335"
1,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",9.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt1375666,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi",8.8,...,True,"$293,662,683","$533,210,699","$826,873,382","$62,785,337 (21.4% of total gross)",4.68 (domestic box office/biggest weekend),35.5% (domestic box office/worldwide),"$160,000,000 (worldwide box office is 5.2 time...","3,792 opening theaters/3,792 max. theaters, 8....","$419,396,244"
3,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139,"Crime,Drama,Thriller",8.8,...,True,NaN,"$105,224","$105,224",NaN,NaN,NaN,NaN,NaN,NaN
4,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994,\N,142,"Drama,Romance",8.8,...,True,"$330,151,138","$349,683,999","$679,835,137","$24,450,602 (7.4% of total gross)",13.50 (domestic box office/biggest weekend),48.6% (domestic box office/worldwide),"$55,000,000 (worldwide box office is 12.4 time...","1,595 opening theaters/2,365 max. theaters, 23...","$909,101,283"


In [29]:
df.drop(columns=['isAdult', 'titleType', 'primaryTitle', 'endYear', 'omdb_Year', 'omdb_Runtime', 'omdb_Genre', 'omdb_imdbRating', 'omdb_imdbVotes', 'omdb_imdbID', 'omdb_Type', 'omdb_DVD', 'omdb_Production', 'omdb_Website'], inplace=True)

In [30]:
df = df.rename(columns={
    'tconst': 'id_imdb',
    'omdb_Title': 'title',
    'startYear': 'year',
    'runtimeMinutes': 'runtime',
    'averageRating': 'imdb_rating',
    'numVotes': 'imdb_votes',
    'omdb_Rated': 'rated',
    'omdb_Released': 'release_date',
    'omdb_Actors': 'actors',
    'omdb_Writer': 'writers',
    'omdb_Director': 'directors',
    'omdb_Plot': 'plot',
    'omdb_Awards': 'awards',
    'omdb_Country': 'country',
    'omdb_Language': 'language',
    'omdb_Poster': 'poster',
    'omdb_Rating': 'ratings',
    'omdb_BoxOffice': 'box_office',
})

In [32]:
df = df[df['omdb_Response'].equals('True')]
df.drop(columns=['omdb_Response'], inplace=True)

KeyError: False

In [ ]:
df.rename(columns={
    'box_office_Production Budget:': 'budget'
}, inplace=True)

,id_imdb,originalTitle,year,runtime,genres,imdb_rating,imdb_votes,title,rated,release_date,...,omdb_Response,box_office_Domestic Box Office,box_office_International Box Office,box_office_Worldwide Box Office,box_office_Opening Weekend:,box_office_Legs:,box_office_Domestic Share:,box_office_Production Budget:,box_office_Theater counts:,box_office_Infl. Adj. Dom. BO
0,tt0111161,The Shawshank Redemption,1994,142,Drama,9.3,3028930.0,The Shawshank Redemption,R,14 Oct 1994,...,True,"$28,313,053","$471,884","$28,784,937","$727,327 (2.6% of total gross)",11.78 (domestic box office/biggest weekend),98.4% (domestic box office/worldwide),"$25,000,000 (worldwide box office is 1.2 times...","33 opening theaters/972 max. theaters, 11.5 we...","$76,238,335"
1,tt0468569,The Dark Knight,2008,152,"Action,Crime,Drama",9.0,3005246.0,The Dark Knight,PG-13,18 Jul 2008,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt1375666,Inception,2010,148,"Action,Adventure,Sci-Fi",8.8,2670018.0,Inception,PG-13,16 Jul 2010,...,True,"$293,662,683","$533,210,699","$826,873,382","$62,785,337 (21.4% of total gross)",4.68 (domestic box office/biggest weekend),35.5% (domestic box office/worldwide),"$160,000,000 (worldwide box office is 5.2 time...","3,792 opening theaters/3,792 max. theaters, 8....","$419,396,244"
3,tt0137523,Fight Club,1999,139,"Crime,Drama,Thriller",8.8,2449965.0,Fight Club,R,15 Oct 1999,...,True,NaN,"$105,224","$105,224",NaN,NaN,NaN,NaN,NaN,NaN
4,tt0109830,Forrest Gump,1994,142,"Drama,Romance",8.8,2367290.0,Forrest Gump,PG-13,06 Jul 1994,...,True,"$330,151,138","$349,683,999","$679,835,137","$24,450,602 (7.4% of total gross)",13.50 (domestic box office/biggest weekend),48.6% (domestic box office/worldwide),"$55,000,000 (worldwide box office is 12.4 time...","1,595 opening theaters/2,365 max. theaters, 23...","$909,101,283"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tt0109445,Clerks,1994,92,Comedy,7.7,236413.0,NaN,NaN,NaN,...,NaN,"$3,073,428","$820,812","$3,894,240",NaN,10.75 (domestic box office/biggest weekend),78.9% (domestic box office/worldwide),"$27,000 (worldwide box office is 144.2 times p...",NaN,"$8,436,887"
996,tt0067992,Willy Wonka & the Chocolate Factory,1971,100,"Adventure,Comedy,Family",7.8,236132.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,tt1142988,The Ugly Truth,2009,96,"Comedy,Romance",6.4,236025.0,NaN,NaN,NaN,...,NaN,"$88,915,214","$116,383,693","$205,298,907","$27,605,576 (31.0% of total gross)",3.22 (domestic box office/biggest weekend),43.3% (domestic box office/worldwide),"$38,000,000 (worldwide box office is 5.4 times...","2,882 opening theaters/2,975 max. theaters, 5....","$134,084,133"
998,tt1980209,Pain & Gain,2013,129,"Action,Biography,Comedy",6.4,235506.0,NaN,NaN,NaN,...,NaN,"$49,875,291","$31,400,000","$81,275,291","$20,244,505 (40.6% of total gross)",2.46 (domestic box office/biggest weekend),61.4% (domestic box office/worldwide),"$26,000,000 (worldwide box office is 3.1 times...","3,277 opening theaters/3,303 max. theaters, 4....","$69,383,706"


In [35]:
df.to_csv("movies_cleaned.csv", sep=";", index=False)

In [34]:


df = df.rename(columns={
    'numVotes': 'num_votes',
    'averageRating': 'average_rating',
    'box_office_Budget': 'budget',
    'box_office_Gross_USA': 'gross_usa',
    'box_office_Gross_Worldwide': 'gross_worldwide',
    'box_office_Open_Weekend': 'open_weekend',
    'box_office_Open_Weekend_USA': 'open_weekend_usa',
    'box_office_Opening_Weekend_USA': 'opening_weekend_usa',
    'box_office_Domestic_Gross': 'domestic_gross'
})
df['budget'] = df['budget'].str.replace('$', '').str.replace(',', '').astype(float)
df['gross_usa'] = df['gross_usa'].str.replace('$', '').str.replace(',', '').astype(float)
df['gross_worldwide'] = df['gross_worldwide'].str.replace('$', '').str.replace(',', '').astype(float)
df['open_weekend'] = df['open_weekend'].str.replace('$', '').str.replace(',', '').astype(float)
df['open_weekend_usa'] = df['open_weekend_usa'].str.replace('$', '').str.replace(',', '').astype(float)

KeyError: 'budget'

In [ ]:
import pygwalker as pyg

Box(children=(HTML(value='\n<div id="ifr-pyg-000632387a9b2decodpiDNO9qKLsHRG1" style="height: auto">\n    <hea…